In [4]:
theano.config.floatX = 'float32'
theano.config.openmp = True

%load_ext autoreload
%autoreload 2


In [20]:
#coding: utf-8 -*-
__doc__="""several more-or-less successfull layer designs used in the lasagne NN architecture"""


import numpy as np
import theano
import theano.tensor as T
import lasagne
floatX = theano.config.floatX
_shared = lambda name,val,dtype: theano.shared(val.astype(dtype),name,
                                               strict = True,allow_downcast=True)

In [31]:
def get_simple_updater(input_shape,n_hidden,name="updater"):
    
    _updater_nn = lasagne.layers.InputLayer(input_shape, name= name+".input")
    
    _updater_nn = lasagne.layers.DenseLayer(_updater_nn,num_units = n_hidden,
                                            nonlinearity=lasagne.nonlinearities.softmax,
                                            name = name+".main")
    
    
    return _updater_nn,_updater_nn.params.items()

In [32]:
upd,params = get_simple_updater([1,8,8],n_hidden=32,name="upd")

In [38]:
pp = params[0][0]
params

[(upd.main.W, {'regularizable', 'trainable'}), (upd.main.b, {'trainable'})]

In [19]:


class MDPLayer(lasagne.layers.Layer):
    def __init__(self,incoming,
                 name="yet_another_mdp",
                 n_units=32,
                 history_length = 10,
                 grad_clipping=0.
                 **kwargs):
        
        super(MDPLayer,self).__init__(incoming,**kwargs)
        
        self.name = name
        self.n_units = n_units
        self.history_length = history_length
        self.grad_clipping = grad_clipping
        
        
        
        inp_shape = list(incoming.output_shape)
        #assumes [single_batch,single_channel,rows,cols]
        assert inp_shape[0] == inp_shape[1] == 1
        
        
        self.hidden_update_layer,params = get_simple_updater([2]+inp_shape[2:],n_units)
        for p_var,p_flags in params:
            self.add_param(p_var,p_flags)
        
        hidden_init = np.zeros(n_units).astype(floatX)
        self.last_hidden = _shared("hidden_state_history",hidden_init,floatX)
        
        
        
    def get_output_shape_for(self, input_shape):
        return (1,self.n_units)
    def get_output_for(self, input, **kwargs):
        
        
        
        if self.grad_clipping:
                gates = theano.gradient.grad_clip(
                    gates, -self.grad_clipping, self.grad_clipping)
        return input

In [ ]:
theano.gradient.grad_clip()